In [12]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [13]:
import pandas as pd
from myUtils import pickle_load, pickle_dump, symb_perf_stats_vectorized
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [14]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [15]:
df = df_OHLCVA[-260::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
import numpy as np
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
symbols_no_volume

Index(['AAIN', 'AAMC', 'ACAB', 'ACAD', 'ACAQ', 'ACBA', 'ACDI', 'ADAL', 'ADER',
       'ADOC',
       ...
       'WLYB', 'WMPN', 'WNNR', 'WQGA', 'WRAC', 'WTMA', 'WWAC', 'XFIN', 'XPER',
       'YNDX'],
      dtype='object', length=527)

In [16]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
df_c
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
symbols_no_close

Index(['AAMC', 'ABM', 'ACAB', 'ACAQ', 'ACDI', 'ACLX', 'ADAL', 'ADRT', 'AEAE',
       'AEHA',
       ...
       'XRAY', 'XYL', 'YNDX', 'YUM', 'ZBH', 'ZBRA', 'ZEUS', 'ZGN', 'ZION',
       'ZNH'],
      dtype='object', length=999)

In [17]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume)
print(f'symbols_drop with duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))
symbols_drop .sort()
print(f'symbols_drop with unique symbols: {len(symbols_drop)}')

symbols_drop with duplicate symbols: 1526
symbols_drop with unique symbols: 1264


In [18]:
df = df.drop(symbols_drop, axis=1, level=0)
df

A                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2021-11-02  157.550003  158.000000  155.440002  157.850006  156.845520   
2021-11-03  158.029999  158.289993  154.570007  158.199997  157.193298   
2021-11-04  157.860001  158.929993  156.360001  157.649994  156.646774   
2021-11-05  156.570007  156.809998  152.429993  156.410004  155.414673   
2021-11-08  157.000000  161.149994  156.550003  160.940002  159.915863   
...                ...         ...         ...         ...         ...   
2022-11-07  136.850006  138.080002  135.559998  138.029999  138.029999   
2022-11-08  138.500000  140.690002  136.600006  138.750000  138.750000   
2022-11-09  138.309998  139.419998  136.660004  137.000000  137.000000   
2022-11-10  142.869995  146.720001  142.529999  146.300003  146.300003   
2022-11-11  147.119995  149.740005  146.350006  148.309998  148.309998   

                              AA                                   ...  \
               Volume       Open       High        Low      Close  ...   
Date                                                               ...   
2021-11-02  1714200.0  46.860001  47.480000  45.900002  47.080002  ...   
2021-11-03  1256500.0  47.200001  48.650002  46.759998  47.430000  ...   
2021-11-04  1015800.0  47.500000  47.619999  45.290001  46.369999  ...   
2021-11-05  1775500.0  46.000000  48.099998  46.000000  47.860001  ...   
2021-11-08  1467000.0  49.200001  50.619999  48.160000  48.860001  ...   
...               ...        ...        ...        ...        ...  ...   
2022-11-07  1213600.0  42.470001  42.790001  41.000000  41.509998  ...   
2022-11-08  1028100.0  42.180000  43.384998  41.459999  43.230000  ...   
2022-11-09   971200.0  42.529999  42.744999  40.340000  40.680000  ...   
2022-11-10  1591900.0  42.810001  45.490002  42.810001  43.830002  ...   
2022-11-11  1227500.0  45.259998  50.759998  45.040001  47.660000  ...   

                 ZUMZ                                        ZWS             \
                  Low      Close  Adj Close    Volume       Open       High   
Date                                                                          
2021-11-02  42.349998  44.290001  44.290001  384000.0  36.060001  36.060001   
2021-11-03  44.139999  47.310001  47.310001  279200.0  35.480000  35.630001   
2021-11-04  47.180000  49.240002  49.240002  387800.0  35.639999  36.029999   
2021-11-05  49.400002  51.480000  51.480000  324900.0  35.720001  35.939999   
2021-11-08  51.040001  51.500000  51.500000  251100.0  35.959999  36.270000   
...               ...        ...        ...       ...        ...        ...   
2022-11-07  22.020000  22.670000  22.670000  172900.0  23.049999  23.260000   
2022-11-08  22.219999  22.469999  22.469999  311800.0  23.170000  23.209999   
2022-11-09  20.639999  20.780001  20.780001  239800.0  22.260000  22.799999   
2022-11-10  21.780001  23.510000  23.510000  327600.0  22.920000  23.910000   
2022-11-11  23.670000  24.280001  24.280001  205000.0  23.590000  24.480000   

                                                        
                  Low      Close  Adj Close     Volume  
Date                                                    
2021-11-02  35.240002  35.560001  35.560001  1665100.0  
2021-11-03  35.099998  35.320000  35.320000  1025600.0  
2021-11-04  35.279999  35.480000  35.480000  1773300.0  
2021-11-05  35.310001  35.720001  35.720001  1344600.0  
2021-11-08  35.689999  35.900002  35.900002   821300.0  
...               ...        ...        ...        ...  
2022-11-07  22.809999  22.959999  22.959999  1587400.0  
2022-11-08  22.340000  22.600000  22.600000  2154600.0  
2022-11-09  21.910000  21.980000  21.980000  1013200.0  
2022-11-10  22.690001  23.440001  23.440001  2877700.0  
2022-11-11  23.549999  24.200001  24.200001  1534600.0  

[260 rows x 14310 columns]

In [19]:
df_close = df_close.iloc[-300::]
df_close = df_close.drop(symbols_drop, axis=1)
df_close

,A,AA,AADI,AAL,AAN,AAON,AAP,AAPL,AAT,AAWW,...,ZIM,ZIP,ZLAB,ZM,ZNTL,ZS,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2021-09-07,176.369461,46.325104,26.480000,19.530001,25.654633,64.327568,193.147842,155.558823,38.598606,73.790001,...,55.099998,24.170000,150.750000,299.959991,68.709999,282.904999,31.320000,207.229309,39.130001,63.119999
2021-09-08,177.371765,45.294991,27.670000,19.129999,25.576950,64.526390,193.505493,153.990234,38.377270,72.769997,...,55.680000,23.660000,149.580002,293.600006,69.720001,280.660004,30.750000,208.745071,38.349998,63.369999
2021-09-09,175.883163,47.058064,27.530001,20.200001,25.916811,64.287804,194.510818,152.957748,37.809502,73.040001,...,58.090000,24.629999,136.990005,295.859985,69.680000,280.220001,31.000000,205.444321,39.599998,62.750000
2021-09-10,175.883163,49.029129,27.920000,18.950001,25.470137,64.685440,193.360504,147.894547,36.750950,74.019997,...,58.610001,25.049999,137.339996,301.500000,69.699997,270.640015,30.760000,206.421585,37.779999,62.820000
2021-09-13,170.772324,48.751797,27.959999,19.309999,25.800287,64.635735,192.616165,148.470367,38.001965,73.209999,...,58.689999,25.570000,137.070007,290.239990,71.870003,267.839996,30.740000,204.427170,38.410000,63.580002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-07,138.029999,41.509998,13.030000,14.320000,9.840000,65.550003,183.089996,138.919998,27.990000,100.440002,...,24.629999,15.090000,32.029999,79.610001,23.549999,117.360001,19.330000,137.860001,22.670000,22.959999
2022-11-08,138.750000,43.230000,13.040000,14.250000,10.120000,74.750000,182.639999,139.500000,27.700001,100.099998,...,24.590000,14.800000,29.830000,78.309998,23.570000,119.930000,19.490000,136.210007,22.469999,22.600000
2022-11-09,137.000000,40.680000,13.410000,13.980000,10.020000,74.540001,175.350006,134.869995,27.219999,99.739998,...,23.170000,14.570000,28.570000,71.839996,18.389999,116.250000,18.680000,133.169998,20.780001,21.980000


In [20]:
print(f'len(symbols_no_volume): {len(symbols_no_volume)}')
print(f'len(symbols_no_close): {len(symbols_no_close)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(symbols_no_volume): 527
len(symbols_no_close): 999
len(symbols_drop): 1264


In [26]:
_periods = [-15, -30, -60, -120]
for _period in _periods:
  f_name = 'file_' + str(_period)


  _df = df_close[_period::]
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(_df)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']

  # write symbols' performance stats to dataframe
  _df = pd.DataFrame(caches_perf_stats_vect, columns=column_names)
  _df['r_Std/UI'] = _df['Std/UI'].rank(ascending=False)
  _df['r_CAGR/Std'] = _df['CAGR/Std'].rank(ascending=False)
  _df['r_CAGR/UI'] = _df['CAGR/UI'].rank(ascending=False)
  _df = _df.sort_values(by=['r_CAGR/UI'])
  pickle_dump(_df, path_data_dump, f'{f_name}')
  print(f_name)

file_-15
file_-30
file_-60
file_-120


In [27]:
df15 = pickle_load(path_data_dump, 'file_-15')
df15

,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI,r_Std/UI,r_CAGR/Std,r_CAGR/UI
309,BAND,2022-10-24,2022-11-11,0.059524,108118.206630,0.052890,2.494440,819506.593428,2.044210e+06,168.0,1.0,1.0
2249,VCYT,2022-10-24,2022-11-11,0.059524,35034.269122,0.024103,3.635235,399841.981365,1.453519e+06,41.0,2.0,2.0
2000,SNCY,2022-10-24,2022-11-11,0.059524,849.402272,0.002990,11.607263,24477.709942,2.841192e+05,5.0,7.0,3.0
2199,TXG,2022-10-24,2022-11-11,0.059524,6914.311770,0.024463,3.589977,78732.820524,2.826490e+05,45.0,3.0,4.0
2179,TSE,2022-10-24,2022-11-11,0.059524,1741.196229,0.018913,2.981237,30880.202451,9.206120e+04,79.0,6.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2081,SWIR,2022-10-24,2022-11-11,0.059524,-0.446979,0.019939,0.316376,-70.855615,-2.241702e+01,2340.0,2384.0,2381.0
1029,GL,2022-10-24,2022-11-11,0.059524,-0.551283,0.021440,0.712508,-36.087205,-2.571243e+01,1733.0,2369.0,2382.0
2248,VCXA,2022-10-24,2022-11-11,0.059524,-0.276007,0.010518,0.522175,-50.254349,-2.624155e+01,2047.0,2380.0,2383.0
1655,ONEM,2022-10-24,2022-11-11,0.059524,-0.219649,0.008011,0.515974,-53.137657,-2.741765e+01,2056.0,2381.0,2384.0
